<a href="https://colab.research.google.com/github/tomonari-masada/course-nlp2020/blob/master/07_document_classification_20201205.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07 単語埋め込みを使った文書分類
* 今回も、IMDbデータセットの感情分析を文書分類問題として解く。
* ただし今回は、fastTextのような学習済みの単語埋め込みは使わない。
* 単語埋め込み自体の学習も、ネットワークの学習と同時におこなう。
* IMDbデータの準備も、`torch.torchtext`を使っておこなう。
 * つまりすべてをPyTorchのなかでおこなう。
* 参考資料
 * https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
 * https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/1%20-%20Simple%20Sentiment%20Analysis.ipynb
 * https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb

## データをどう扱うか
* ネットワークへの入力は、単語埋め込みを、単語の出現順どおりに並べた列にする。
 * ミニバッチは[ミニバッチのなかでの最大文書長, ミニバッチのサイズ, 単語埋め込み次元数]という形の3階のテンソルになる。
* そして、前向き計算のなかではじめて、単語埋め込みの平均をとることにする。
 * `.mean(0)`と、軸0で平均をとることになる。


## 07-00 Google Colabのランタイムのタイプを変更する
* Google ColabのランタイムのタイプをGPUに変更しておこう。
 * 上のメニューの「ランタイム」→「ランタイムのタイプを変更」→「ハードウェア　アクセラレータ」から「GPU」を選択

## 07-01 torchtextを使ってIMDbデータを読み込む
* ここでIMDbデータセットの読み込みにつかう`torchtext.datasets`については、下記を参照。
 * https://torchtext.readthedocs.io/en/latest/datasets.html

### 実験の再現性確保のための設定など
* https://pytorch.org/docs/stable/notes/randomness.html

In [1]:
import random
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data
from torchtext import datasets
from torchtext.data import Field, LabelField, BucketIterator

SEED = 123

random.seed(SEED)
torch.manual_seed(SEED)
torch.set_deterministic(True)

### torchtextのフィールド
* TEXTフィールドと、LABELフィールドという２種類のFieldオブジェクトのインスタンスを作る。
 * Fieldクラスの詳細については[ここ](https://github.com/pytorch/text/blob/master/torchtext/data/field.py)を参照。
* TEXTフィールドは、テキストの前処理の仕方を決めておくのに使う。
 * tokenizerは、デフォルトでは単にstring型のsplitメソッドを適用するだけになる。これは高速だが、tokenizationとしては雑。
* LABELフィールドは、ラベルの前処理に使う。

In [2]:
TEXT = Field(tokenize="spacy")
LABEL = LabelField()

### IMDbデータセットをダウンロードした後、前処理しつつ読み込む
* ダウンロードはすぐ終わるが、解凍に少し時間がかかる。
* また、TEXTフィールドでspaCyのtokenizationを使うように設定したので、少し時間がかかる。
 * string型のsplitメソッドでtokenizeすると、時間はあまりかからない。（そのかわり、やや雑なtokenizationになる。）

In [3]:
train_valid_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:03<00:00, 21.2MB/s]


### 最初の文書を見てみる

In [4]:
print(train_valid_data[0].text)

['What', 'could', 'be', 'more', 'schlocky', 'than', 'the', 'idea', 'of', 'private', 'detectives', 'getting', 'involved', 'with', 'the', 'women', 'they', "'re", 'supposed', 'to', 'be', 'spying', 'on', '?', 'And', 'most', 'of', 'the', 'dialogue', 'as', 'written', 'is', 'perfectly', 'banal.<br', '/><br', '/>But', 'the', 'actors', 'turn', 'the', 'dialog', 'into', 'something', 'that', 'makes', 'sense', '.', 'You', 'can', 'see', 'real', 'people', 'behind', 'the', 'unreal', 'lines', '.', 'And', 'the', 'directing', 'is', 'wonderful', '.', 'Each', 'scene', 'does', 'just', 'what', 'it', 'has', 'to', 'and', 'ends', 'without', 'dragging', 'on', 'too', 'long.<br', '/><br', '/>I', 'showed', 'this', 'to', 'several', 'friends', 'in', 'the', 'mid-80s', 'because', 'I', 'was', 'perplexed', 'at', 'how', 'such', 'bad', 'material', 'could', 'be', 'made', 'into', 'such', 'a', 'good', 'movie', '.', 'The', 'friends', 'enjoyed', 'it', 'too', '.']


In [5]:
print(train_valid_data[0].label)

pos


### テストセット以外の部分を訓練データと検証データに分ける

In [7]:
train_data, valid_data = train_valid_data.split(split_ratio=0.8)

In [8]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 20000
Number of validation examples: 5000
Number of testing examples: 25000


In [9]:
print(train_data[0].text)

['I', 'was', 'pleasantly', 'surprised', 'by', 'the', 'film', '.', 'Let', "'s", 'face', 'it', ';', 'the', 'premise', 'does', "n't", 'sound', 'particularly', 'appealing', 'when', 'having', 'to', 'hand', 'over', 'money', 'for', 'a', 'the', 'night', "'s", 'flick', ',', 'but', 'it', 'had', 'an', 'easygoing', 'nature', 'that', 'wins', 'one', 'over', '.', 'There', 'were', 'no', 'moments', 'that', 'I', 'found', 'uproarious', ',', 'and', 'I', 'doubt', 'any', 'that', 'I', "'ll", 'remember', 'the', 'next', 'day', ',', 'but', 'this', 'does', "n't", 'fail', 'as', 'a', 'nice', 'diversion', '.', 'What', 'I', 'found', 'funny', 'was', 'watching', 'it', 'here', 'in', 'Peking', 'with', 'my', 'Chinese', 'girlfriend', 'who', 'never', 'understands', 'anything', 'I', 'like', '.', 'I', 'told', 'her', 'there', 'was', 'a', 'plot-', 'three', 'guys', 'have', 'to', 'bring', 'back', 'some', 'weed', 'to', 'London', '.', 'Hardly', 'satisfying', 'for', 'her', '.', 'There', 'is', 'no', 'mugging', 'going', 'on', 'here',

### データセットの語彙とラベルを作る
* TEXTラベルのほうでは、最大語彙サイズを指定する。

In [10]:
MAX_VOCAB_SIZE = 25000 # この値は適当。

TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

なぜ語彙サイズが25,000ではなく25,002なのかについては、少し下の説明を参照。

In [11]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


### 出現頻度順で上位２０単語を見てみる

In [12]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 232468), (',', 220692), ('.', 190092), ('a', 125458), ('and', 125391), ('of', 115472), ('to', 107502), ('is', 87667), ('in', 70419), ('I', 62409), ('it', 61426), ('that', 56428), ('"', 50636), ("'s", 49514), ('this', 48315), ('-', 42246), ('/><br', 40827), ('was', 39860), ('as', 34976), ('with', 34256)]


### 単語ID順に最初の１０単語を見てみる
* IDのうち、0と1は、未知語とパディング用の単語という特殊な単語に割り振られている。
 * 未知語は`<unk>`という特殊な単語に置き換えられる。これのIDが0。
 * パディングとは、長さが不揃いの複数の文書を同じミニバッチにまとめるとき、すべての文書の長さを無理やりそろえるため、文書末尾に特殊な単語（元々の語彙にない、人工的に用意した単語）を追加すること。
 * パディング用の単語が`<pad>`になっているのは、上のほうで使ったFieldクラスのインスタンスを作るときのデフォルトの値がこの`<pad>`になっているため。

In [13]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']


### ラベルのほうのIDを確認する
* こちらはnegとposに対応する２つのIDしかない。

In [15]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7faa4ac121e0>, {'neg': 0, 'pos': 1})


### ミニバッチを取り出すためのiteratorを作る
* ミニバッチのサイズを指定する。
 * ミニバッチのサイズは、性能を出すためにチューニングすべきハイパーパラメータのひとつ。

In [20]:
BATCH_SIZE = 100

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator = data.BucketIterator(train_data, batch_size=BATCH_SIZE, device=device,
                                     sort_within_batch=True, shuffle=True, sort_key=lambda x: len(x.text))
valid_iterator = data.BucketIterator(valid_data, batch_size=BATCH_SIZE, device=device)
test_iterator = data.BucketIterator(test_data, batch_size=BATCH_SIZE, device=device)

### ミニバッチの中身を確認する

* 訓練データのiteratorを回してミニバッチをすべて取得してみる
 * ミニバッチのshapeを表示してみる

In [21]:
for i, batch in enumerate(train_iterator):
  print(i, batch.text.shape)

0 torch.Size([132, 100])
1 torch.Size([462, 100])
2 torch.Size([251, 100])
3 torch.Size([214, 100])
4 torch.Size([106, 100])
5 torch.Size([386, 100])
6 torch.Size([95, 100])
7 torch.Size([188, 100])
8 torch.Size([291, 100])
9 torch.Size([113, 100])
10 torch.Size([50, 100])
11 torch.Size([544, 100])
12 torch.Size([191, 100])
13 torch.Size([162, 100])
14 torch.Size([229, 100])
15 torch.Size([135, 100])
16 torch.Size([395, 100])
17 torch.Size([832, 100])
18 torch.Size([182, 100])
19 torch.Size([170, 100])
20 torch.Size([246, 100])
21 torch.Size([929, 100])
22 torch.Size([1054, 100])
23 torch.Size([71, 100])
24 torch.Size([304, 100])
25 torch.Size([273, 100])
26 torch.Size([335, 100])
27 torch.Size([218, 100])
28 torch.Size([623, 100])
29 torch.Size([128, 100])
30 torch.Size([279, 100])
31 torch.Size([226, 100])
32 torch.Size([155, 100])
33 torch.Size([480, 100])
34 torch.Size([65, 100])
35 torch.Size([175, 100])
36 torch.Size([407, 100])
37 torch.Size([447, 100])
38 torch.Size([121, 100])

* ミニバッチの形は、[ミニバッチ内での最大文書長, ミニバッチのサイズ]になっていることに注意！
 * ミニバッチのサイズが最初に来ているのではない！
 * [ミニバッチのサイズ, ミニバッチ内での最大文書長]という形にしたいなら、テキストのfieldを作るとき以下のようにする。

__`TEXT = data.Field(tokenize="spacy", batch_first=True)`__

* 上記のループを抜けたあとには、変数batchには最後のミニバッチが代入されている。

In [22]:
batch.text.shape

torch.Size([140, 100])

* このミニバッチに含まれる文書のうち、最初の文書の単語ID列を表示させてみる。

In [23]:
print(batch.text[:, 0])

tensor([  501,  3773,     2,    24,     3,    14,    56,    15,   146,    33,
            8,   137,     4,  5846,   157,     8,  1002,     4,  1263,     7,
           42,     8,    36,  2781,     4,    14,   741,     3,     2,     0,
           18,   890,  1668,    74,    21,    42,     2,  1836,     0,     7,
            5,   363,   221,    24,     3,    26,     0,    96,   434,   139,
            2,   106,  7428,     7,     2,    24,     3,     6,    31,   253,
           47,    31,   198,   130, 20297,     8,    45,     2, 15686,    18,
          409,    15,    20,    72,     2,   942,    50,     0,   121,   434,
           69,    36,  8870,    64,    41,    96,  1120,   269,     6,     5,
         1527,  6545,   141,     4,     0,     3,   457,     3,    22,     5,
           24,    47,    25,     0,     3,    33,     5,  5099,    17,   291,
          221,    24,     4,    25,   953,     9,    13,     5,  4846,   103,
          971,     6,    59,  4846,   103,   661,   158,   167, 

* このミニバッチに含まれる文書のうち、最初の文書の単語ID列を単語列に戻したものを表示させてみる。

In [24]:
print(' '.join([TEXT.vocab.itos[i] for i in batch.text[:, 0]]))

To quote the film , " It 's better not to know . Better still to forget . Best of all to be abandoned . " Oh , the <unk> /><br />A ghost story with all the technical <unk> of a Hollywood horror film , but <unk> bad dialogue after the first quarter of the film , and you feel like you 're being preached to from the start.<br /><br />It 's as if the writers ' <unk> character dialogue can be summed up by bad cop TV and a Jerry Springer show . <unk> , maybe , for a film like The <unk> , not a Russia - set horror film . The result is that a potentially great setting and some potentially great gore scenes go to waste and become just silly , not scary or meaningful .


* このミニバッチに含まれる文書のうち、最後の文書の単語ID列を表示させてみる。

In [25]:
print(batch.text[:, BATCH_SIZE-1])

tensor([  155,   442,    16,    23,     5,   192,     4,    11,  1654,    16,
         1050,   163,    33,   116,    96,     8,  1156,    43,  2736,     4,
           56,   679,    55,     5,   616,  2481,   621,     4,   594,   158,
           79,   716,     6,  1349,     8,    36,    48,    10,     5,    23,
           33,  1468,     8,    36,    54,     5,   416,  2829,   238,    26,
           21,    59,  1252,     6,  1676,  1277,     4,    56,  1035,    30,
          245,  5874,     4,    25,   861,    32,    42,    60,    26,   295,
            2,   262,    44,   420,     3,   108,  1084,     6,  1096,     2,
           23,     4, 15229,   133,   125,   115,     7,   169,   175,   188,
            3,    26,    46,    79,  1458,     6,   105,    84,   497,    53,
           65,   598,     8,   109,    10,     2,   150,    70,     9,   684,
            5,   192,     4,    25,   251,     9,  1135,     6,  1228,     6,
           52,     5,  2569,   253,     8,    12,     4,    11, 

最後の文書の末尾は「1」で埋められていることが分かる。

この1は、パディング用単語のIDだったことを想起されたい。

ミニバッチに含まれる文書の長さを調べると、文書が文書長の降順に並べられていることが分かる。

In [26]:
(batch.text != 1).sum(0)

tensor([140, 140, 140, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139,
        139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139,
        139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139,
        139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139,
        139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 138, 138, 138,
        138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138,
        138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138,
        138, 138], device='cuda:0')

## 07-02 MLPによる文書分類の準備
* 今回は、ごく簡単なMLPで文書分類をする。
* 文書中の全単語トークンの埋め込みベクトルの平均を、MLPの入力とする。
 * 当然、語順の情報は使われない。
 * つまり、bag-of-wordsモデルになっている。

### 定数の設定
* 単語埋め込みベクトルの次元数は128にする。

In [28]:
INPUT_DIM = len(TEXT.vocab)
NUM_CLASS = len(LABEL.vocab)
EMBED_DIM = 128
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

print(f'語彙サイズ {INPUT_DIM}, クラス数 {NUM_CLASS}, 単語埋め込み次元 {EMBED_DIM}')

語彙サイズ 25002, クラス数 2, 単語埋め込み次元 128


### モデルを定義する前にPyTorchの単語埋め込みがどんなものかを見てみる
* 埋め込みとは、単語IDから単語ベクトルへのマッピング。

* 以下のように、語彙サイズと埋め込みの次元数を指定しつつ、torch.nn.Embeddingのインスタンスを作ればよい。

In [33]:
embed = nn.Embedding(INPUT_DIM, EMBED_DIM, padding_idx=PAD_IDX).to(device)

* パディング用の単語の埋め込みはゼロベクトルになる。

In [31]:
print(embed(torch.tensor([21,1])))

tensor([[ 6.8744e-02,  4.2849e-01,  2.1481e-01,  8.7952e-01, -1.7290e+00,
         -2.2350e+00, -5.4192e-01, -1.3365e+00, -2.2437e+00,  1.9041e+00,
          2.1003e+00,  2.1210e+00, -8.0829e-02, -8.6538e-02, -2.0858e-01,
         -3.8659e-01,  1.2082e+00, -5.3790e-01, -7.1226e-01,  9.7439e-01,
         -1.2953e+00,  1.4643e+00, -5.7980e-01,  1.0861e+00, -7.8486e-01,
         -1.2188e+00, -1.7692e+00,  1.5884e+00,  2.3673e+00, -1.7137e+00,
         -8.2795e-03,  5.4360e-01, -2.5634e-01,  1.0775e+00,  7.8097e-01,
          1.2306e-03,  6.7369e-01,  1.8175e+00,  7.9229e-01,  1.9029e-01,
          2.2103e+00, -2.0936e+00,  1.4660e+00, -5.3831e-01, -1.3515e+00,
          4.0773e-01,  8.7801e-01, -1.2269e+00, -8.7526e-01,  1.6439e+00,
          3.0164e+00,  1.0737e+00,  1.7512e+00, -1.8375e+00,  8.7735e-01,
         -1.5085e-01,  2.1493e+00, -3.1661e-01, -1.0365e+00, -1.1609e+00,
         -6.0594e-01, -1.3657e+00,  4.2652e-01,  3.7043e-01, -3.0607e-01,
         -7.3270e-01, -1.6886e+00,  3.

* 埋め込みの効果を見てみる

In [34]:
for i, batch in enumerate(train_iterator):
  print(i, embed(batch.text).shape)

0 torch.Size([315, 100, 128])
1 torch.Size([212, 100, 128])
2 torch.Size([435, 100, 128])
3 torch.Size([209, 100, 128])
4 torch.Size([194, 100, 128])
5 torch.Size([365, 100, 128])
6 torch.Size([146, 100, 128])
7 torch.Size([143, 100, 128])
8 torch.Size([178, 100, 128])
9 torch.Size([81, 100, 128])
10 torch.Size([1054, 100, 128])
11 torch.Size([165, 100, 128])
12 torch.Size([239, 100, 128])
13 torch.Size([191, 100, 128])
14 torch.Size([247, 100, 128])
15 torch.Size([2789, 100, 128])
16 torch.Size([139, 100, 128])
17 torch.Size([141, 100, 128])
18 torch.Size([338, 100, 128])
19 torch.Size([386, 100, 128])
20 torch.Size([222, 100, 128])
21 torch.Size([216, 100, 128])
22 torch.Size([206, 100, 128])
23 torch.Size([294, 100, 128])
24 torch.Size([63, 100, 128])
25 torch.Size([57, 100, 128])
26 torch.Size([528, 100, 128])
27 torch.Size([288, 100, 128])
28 torch.Size([276, 100, 128])
29 torch.Size([181, 100, 128])
30 torch.Size([931, 100, 128])
31 torch.Size([257, 100, 128])
32 torch.Size([171,

### モデルの定義
* MLP（多層パーセプトロン）だが、入り口に単語埋め込み層が挿入されている。

In [35]:
class EmbedTextSentiment(nn.Module):
  def __init__(self, embed_dim, num_class, vocab_size, padding_idx):
    super(EmbedTextSentiment, self).__init__()
    self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=padding_idx)
    self.fc1 = nn.Linear(embed_dim, 500)
    self.fc2 = nn.Linear(500, 100)
    self.fc3 = nn.Linear(100, num_class)

  def forward(self, text):
    x = self.embed(text)
    x = x.mean(0) # 文書に含まれる全単語トークンの単語ベクトルの平均
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

### モデルを作る
* モデル（のインスタンス）をGPUに移動させている点に注意。

In [36]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX).to(device)

### 損失関数とoptimizerとschedulerを作る

In [38]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

### 訓練用の関数
* 最初の`model.train()`に注意。こうやって、モデルを訓練モードに設定する。
 * 例えば、dropoutを含むモデルなど、訓練時と評価時で、ふるまい方を変える必要があるときがあるため、こういうことをする。

In [39]:
def train(data_iterator, model, optimizer, scheduler, criterion):

  model.train()

  train_loss = 0
  train_acc = 0
  for batch in data_iterator:
    optimizer.zero_grad()
    text, cls = batch.text, batch.label
    output = model(text)
    loss = criterion(output, cls)
    train_loss += loss.item()
    loss.backward()
    optimizer.step()
    train_acc += (output.argmax(1) == cls).float().mean().item()

  scheduler.step()

  num_batch = len(data_iterator)
  return train_loss / num_batch, train_acc / num_batch

### 評価用の関数
* 最初の`model.eval()`に注意。こうやって、モデルを評価モードに設定する。
 * 例えば、dropoutを含むモデルなど、訓練時と評価時で、ふるまい方を変える必要があるときがあるため、こういうことをする。

In [40]:
def test(data_iterator, model, criterion):

  model.eval()

  loss = 0
  acc = 0
  for batch in data_iterator:
    text, cls = batch.text, batch.label
    with torch.no_grad():
      output = model(text)
      loss = criterion(output, cls)
      loss += loss.item()
      acc += (output.argmax(1) == cls).float().mean().item()

  num_batch = len(data_iterator)
  return loss / num_batch, acc / num_batch

## 07-03 分類器の訓練と検証セットでの評価

In [41]:
N_EPOCHS = 20
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  valid_loss, valid_acc = test(valid_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs // 60
  secs = secs % 60

  print(f'Epoch {epoch + 1} | time in {mins:d} minutes, {secs:d} seconds | ', end='')
  for param_group in optimizer.param_groups:
    print(f'lr={param_group["lr"]:.6f}')
    break
  print(f'\tLoss: {train_loss:.5f}(train)\t|\tAcc: {train_acc * 100:.2f}%(train)')
  print(f'\tLoss: {valid_loss:.5f}(valid)\t|\tAcc: {valid_acc * 100:.2f}%(valid)')

Epoch 1 | time in 0 minutes, 2 seconds | lr=0.009500
	Loss: 0.38066(train)	|	Acc: 82.44%(train)
	Loss: 0.02088(valid)	|	Acc: 80.12%(valid)
Epoch 2 | time in 0 minutes, 2 seconds | lr=0.009025
	Loss: 0.16610(train)	|	Acc: 93.88%(train)
	Loss: 0.01925(valid)	|	Acc: 87.10%(valid)
Epoch 3 | time in 0 minutes, 2 seconds | lr=0.008574
	Loss: 0.07470(train)	|	Acc: 97.33%(train)
	Loss: 0.01066(valid)	|	Acc: 87.26%(valid)
Epoch 4 | time in 0 minutes, 2 seconds | lr=0.008145
	Loss: 0.04084(train)	|	Acc: 98.58%(train)
	Loss: 0.01672(valid)	|	Acc: 83.78%(valid)
Epoch 5 | time in 0 minutes, 2 seconds | lr=0.007738
	Loss: 0.02609(train)	|	Acc: 99.02%(train)
	Loss: 0.01654(valid)	|	Acc: 86.76%(valid)
Epoch 6 | time in 0 minutes, 2 seconds | lr=0.007351
	Loss: 0.01718(train)	|	Acc: 99.40%(train)
	Loss: 0.01959(valid)	|	Acc: 85.54%(valid)
Epoch 7 | time in 0 minutes, 2 seconds | lr=0.006983
	Loss: 0.01117(train)	|	Acc: 99.62%(train)
	Loss: 0.01200(valid)	|	Acc: 83.84%(valid)
Epoch 8 | time in 0 minutes

## 07-04 再検討
* 訓練データ上での分類精度がほぼ100%になってしまっている。
* 検証データでの分類精度と大きな差があり、明らかにオーバーフィッティング。

### ドロップアウトを使う
* モデルのインスタンスを作るときにdropoutの確率を引数pで指定できるようにする。

In [ ]:
class EmbedTextSentiment(nn.Module):
  def __init__(self, embed_dim, num_class, vocab_size, padding_idx, p=0.0):
    super(EmbedTextSentiment, self).__init__()
    self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=padding_idx)
    self.dropout = nn.Dropout(p=p)
    self.fc1 = nn.Linear(embed_dim, 500)
    self.fc2 = nn.Linear(500, 100)
    self.fc3 = nn.Linear(100, num_class)

  def forward(self, text):
    x = self.dropout(self.embed(text)) #埋め込み層の直後にdropout
    x = x.mean(0)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [ ]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX, p=0.5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

In [ ]:
N_EPOCHS = 20
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  valid_loss, valid_acc = test(valid_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs // 60
  secs = secs % 60

  print(f'Epoch {epoch + 1} | time in {mins:d} minutes, {secs:d} seconds | ', end='')
  for param_group in optimizer.param_groups:
    print(f'lr={param_group["lr"]:.6f}')
    break
  print(f'\tLoss: {train_loss:.5f}(train)\t|\tAcc: {train_acc * 100:.2f}%(train)')
  print(f'\tLoss: {valid_loss:.5f}(valid)\t|\tAcc: {valid_acc * 100:.2f}%(valid)')

### L２正則化を使う
* optimizerのweight_decayパラメータを0より大きな値にする。

In [ ]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX, p=0.5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

In [ ]:
N_EPOCHS = 20
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  valid_loss, valid_acc = test(valid_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs // 60
  secs = secs % 60

  print(f'Epoch {epoch + 1} | time in {mins:d} minutes, {secs:d} seconds | ', end='')
  for param_group in optimizer.param_groups:
    print(f'lr={param_group["lr"]:.6f}')
    break
  print(f'\tLoss: {train_loss:.5f}(train)\t|\tAcc: {train_acc * 100:.2f}%(train)')
  print(f'\tLoss: {valid_loss:.5f}(valid)\t|\tAcc: {valid_acc * 100:.2f}%(valid)')

### early stopping
* dev setでのaccuracyが4回連続で最高値を下回ったら訓練を終えることにする。
* early stoppingの実現については、PyTorch Lightningを使う手もある。
 * https://pytorch-lightning.readthedocs.io/en/latest/early_stopping.html

In [ ]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX, p=0.5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

In [ ]:
patience = 4
early_stop_count = 0
best_valid_acc = 0.0

MIN_N_EPOCHS = 10 # 最低このエポック数は実行する
N_EPOCHS = 50 # エポック数を増やしておく
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  valid_loss, valid_acc = test(valid_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs // 60
  secs = secs % 60

  print(f'Epoch {epoch + 1} | time in {mins:d} minutes, {secs:d} seconds | ', end='')
  for param_group in optimizer.param_groups:
    print(f'lr={param_group["lr"]:.6f}')
    break
  print(f'\tLoss: {train_loss:.5f}(train)\t|\tAcc: {train_acc * 100:.2f}%(train)')
  print(f'\tLoss: {valid_loss:.5f}(valid)\t|\tAcc: {valid_acc * 100:.2f}%(valid)')

  # early stopping
  if epoch + 1 > MIN_N_EPOCHS:
    if best_valid_acc <= valid_acc:
      best_valid_acc = valid_acc
      early_stop_count = 0
    else:
      early_stop_count += 1
      if early_stop_count == patience:
        break

## 07-05 テストセット上で評価
* 見つけ出したベストな設定を使って、テストセット上での最終的な評価をおこなう。

In [ ]:
print('Checking the results of test dataset...')
test_loss, test_acc = test(test_iterator, model, criterion)
print(f'\tLoss: {test_loss:.5f}(test)\t|\tAcc: {test_acc * 100:.2f}%(test)')